# Some EDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Energy = pd.read_csv('../../data/energy_dataset.csv')
Energy = Energy[Energy['total load actual'].notna()]
Weather = pd.read_csv('AllTemps.csv', skipinitialspace=True)

print(list(Energy.keys()))
print(list(Weather.keys()))
print(Weather.shape)
print(Energy.shape)

['time', 'generation biomass', 'generation fossil brown coal/lignite', 'generation fossil coal-derived gas', 'generation fossil gas', 'generation fossil hard coal', 'generation fossil oil', 'generation fossil oil shale', 'generation fossil peat', 'generation geothermal', 'generation hydro pumped storage aggregated', 'generation hydro pumped storage consumption', 'generation hydro run-of-river and poundage', 'generation hydro water reservoir', 'generation marine', 'generation nuclear', 'generation other', 'generation other renewable', 'generation solar', 'generation waste', 'generation wind offshore', 'generation wind onshore', 'forecast solar day ahead', 'forecast wind offshore eday ahead', 'forecast wind onshore day ahead', 'total load forecast', 'total load actual', 'price day ahead', 'price actual']
['Unnamed: 0', 'time', 'tempV', 'tempM', 'tempBi', 'tempBa', 'tempS', 'humV', 'humM', 'humBi', 'humBa', 'humS', 'cldV', 'cldM', 'cldBi', 'cldBa', 'cldS', 'wndV', 'wndM', 'wndBi', 'wndBa'

Made sure that both dataframes are shaped the same

### Let's try to make a dataframe with the averages of the population, and an integer representation of time for input into a Neural Network

In [2]:
pops2017 = {"V": 788000, "M": 3183000, "Bi": 345000, "Ba": 1621000, "S": 689000}
totalPop2017= 788000 + 3183000 + 345000 + 1621000 + 689000
pops2017["V"] /= totalPop2017
pops2017["M"] /= totalPop2017
pops2017["Bi"] /= totalPop2017
pops2017["Ba"] /= totalPop2017
pops2017["S"] /= totalPop2017

def WeightedAverages(Wvar, row, pops):
    return (pops["V"] * row[Wvar + "V"]) + (pops["M"] * row[Wvar + "M"]) + (pops["Bi"] * row[Wvar + "Bi"]) + (pops["Ba"] * row[Wvar + "Ba"]) + (pops["S"] * row[Wvar + "S"])

def IsolateHour(Time):
    Time = Time.split(" ")
    Time = Time[1].split(":")
    return Time[0]

IntTimes = []
TempsAverage = []
HumAvererage = []
CldAverage = []
WndAverage = []
for index, row in Weather.iterrows():
    TempsAverage.append(WeightedAverages("temp", row, pops2017))
    HumAvererage.append(WeightedAverages("hum", row, pops2017))
    CldAverage.append(WeightedAverages("cld", row, pops2017))
    WndAverage.append(WeightedAverages("wnd", row, pops2017))
    IntTimes.append(IsolateHour(row["time"]))


WxAverages = pd.DataFrame({
    "time": np.asarray(IntTimes).astype('float64'),
    "tAve": np.asarray(TempsAverage).astype('float64'),
    "hAve": np.asarray(HumAvererage).astype('float64'),
    "cAve": np.asarray(CldAverage).astype('float64'),
    "wAve": np.asarray(WndAverage).astype('float64')
})


In [3]:
# MinMax Scale the numerical features
colScales = []
colMins = []
def MinMaxScaler(column):
    scale = max(column)-min(column)
    colScales.append(scale)
    colMins.append(min(column))
    return (column-min(column))/scale
    
for col in WxAverages.columns:
    WxAverages[col] =   MinMaxScaler(WxAverages[col])

y = MinMaxScaler(Energy['total load actual'])

print(y.isnull().any())
print(colScales)
print(colMins)

NNData = pd.concat([WxAverages, pd.DataFrame(data=y, columns = ["load"])], axis=1)
NNData.to_csv('NNData.csv', index=False)

False
[23.0, 40.115214679859605, 79.76803501358285, 96.34228795653486, 20.70917597343797, 22974.0]
[0.0, 269.77175332496984, 19.867491699366134, 0.0, 0.0, 18041.0]


### Now to actually do the NN

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics as tfmetrics
import tensorflow as tf

def create_model(h1=3, h2=3):
    model = Sequential(name='classifier')

    layers = [
        Dense(5, activation='elu', name='input', kernel_initializer="normal"),
        Dense(h1, activation='elu', name='h1', kernel_initializer="normal"),
        Dense(h2, activation='elu', name='h2', kernel_initializer="normal"),
        Dense(1, activation='tanh', name='out', kernel_initializer="normal")
    ]

    for layer in layers:
        model.add(layer)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=[tfmetrics.RootMeanSquaredError()])

    return model

model = create_model()
pricemodel = create_model()

from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(WxAverages, y, test_size = 0.1, random_state=0)
print(os.getenv("CUDA_VISIBLE_DEVICES"))

with tf.device("/cpu:0"):
    hist = model.fit(Xtrain.to_numpy(), ytrain.to_numpy(), epochs=500, verbose=1)


2022-06-02 04:07:38.558606: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-02 04:07:38.558647: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-06-02 04:07:39.555053: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-06-02 04:07:39.811204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-02 04:07:39.813647: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 10 deviceMemorySize:

None
Epoch 1/500


2022-06-02 04:07:40.136505: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-02 04:07:40.136814: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2208010000 Hz


986/986 [==============================] - 2s 1ms/step - loss: 0.0476 - root_mean_squared_error: 0.2183
Epoch 2/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0261 - root_mean_squared_error: 0.1617
Epoch 3/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0261 - root_mean_squared_error: 0.1615
Epoch 4/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0260 - root_mean_squared_error: 0.1612
Epoch 5/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0258 - root_mean_squared_error: 0.1607
Epoch 6/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0257 - root_mean_squared_error: 0.1603
Epoch 7/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0255 - root_mean_squared_error: 0.1597
Epoch 8/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0253 - root_mean_squared_error: 0.1591
Epoch 9/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0250 - 

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = model.predict(Xtest, verbose=0)
MSE = mean_squared_error(y_true=ytest, y_pred=y_pred)
RMSE = mean_squared_error(y_true=ytest, y_pred=y_pred, squared=False)
r2 = r2_score(y_true=ytest, y_pred=y_pred)
print("Load Scores:")
print('Testing MSE: %.5f' % MSE)
print('Testing RMSE: %.5f' % RMSE)
print("r^2 score: %.5f" % r2)

Load Scores:
Testing MSE: 0.01864
Testing RMSE: 0.13654
r^2 score: 0.53205


In [6]:
model.save('LoadNN.ann')

2022-06-02 04:17:08.432201: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: LoadNN.ann/assets
